### Dependencies:
Uncomment the below code and run cell to install all dependencies. It is recommended to create a new virtual environment. This is done using conda and python 3.11.


In [2]:
# !pip install pypyodbc
# !pip install polars
# !pip install bs4
# !pip install selenium

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pypyodbc: filename=pypyodbc-1.3.6-py3-none-any.whl size=22871 sha256=230a512de1136ba601bdbcabc866f833c996c13ed79b97d9a50dbaf5eb5158ff
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\5d\fa\2f\becd808255e1b61b755eb3a3aa256229a7d7e1fc89e2aef2bc
Successfully built pypyodbc
   ---------------------------------------- 0.0/31.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/31.5 MB ? eta -:--:--
    --------------------------------------- 0.8/31.5 MB 3.0 MB/s eta 0:00:11
   - -------------------------------------- 1.0/31.5 MB 2.0 MB/s eta 0:00:16
   - -------------------------------------- 1.6/31.5 MB 2.3 MB/s eta 0:00:14
   -- ------------------------------------- 2.4/31.5 MB 2.6 MB/s eta 0:00:12
   --- ------------------------------------ 2.9/31.5 MB 2.8 MB/s eta 0:00:11
   --- ------------------------------------ 3.1/31.5 MB 2.6 MB/

In [3]:
import pypyodbc as odbc

## Base work:
Familiarising myself with the code so that, in future, I can make modules.
Below, I am connecting to the database, this is done on my desktop PC - I am practicing this now!

In [4]:
DRIVER_NAME = 'SQL Server'
# SERVER_NAME = 'LAPTOP-79UCG6D3\SQLEXPRESS'
SERVER_NAME = 'DESKTOP-Q3IFECL\SQLEXPRESS01'
DATABASE_NAME = 'UFC-STATS'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trusted_Connection=yes;

"""

con = odbc.connect(connection_string)

print(con)

## Selenium
Selenium is a popular open-source framework used for automating web browsers. It provides a set of tools and libraries that allow developers to interact with web elements, simulate user actions, and perform automated testing of web applications.
One of the key features of Selenium is its ability to locate and interact with web elements on a page using various methods such as XPath, CSS selectors, and element attributes. This enables you to perform actions like clicking buttons, entering text, selecting dropdown options, and verifying the presence of specific elements.

In addition to automated testing, Selenium can be used for web scraping, data extraction, and web application monitoring. It allows you to retrieve data from websites, scrape information, and perform tasks at regular intervals to monitor the behavior and performance of web applications.
Overall, Selenium is a powerful tool for automating web browsers and performing various tasks related to web testing, web scraping, and web application monitoring. Its flexibility, cross-browser compatibility, and extensive community support make it a popular choice among developers and testers.

### Part 1: Get all Events 
The first page of the ufc stats page includes event names for all ufc events, including ufc fight nights.

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import bs4

# PATH = 'C:\Program Files (x86)\chromedriver.exe'

driver = webdriver.Chrome()

driver.get('http://ufcstats.com/statistics/events/completed')


soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')

page_links = soup.find_all('a', class_='b-link b-link_style_black') 


driver.quit()

In [4]:
print(soup.prettify())

<html class="js flexbox canvas canvastext webgl no-touch geolocation postmessage no-websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" style="">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   Stats | UFC
  </title>
  <meta content="" name="description"/>
  <meta content="" name="viewport"/>
  <link href="/blocks/main.css?ver=574860" rel="stylesheet"/>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-69CJ0D6450&amp;cx=c&amp;_slc=1" type="text/javascript">
  </script>
  <script async="" src="//www.google-analytics.com/analytics.js">
  </script>
  <script src="/js/v

In [5]:
dates_source = soup.find_all('span', class_='b-statistics__date')
event_dates = [date.get_text(strip=True) for date in dates_source]
event_dates = event_dates[1:]
event_names = [link.get_text(strip=True) for link in page_links]

## Polars - a more efficent alternative to the widely used Pandas
This works very similar to pandas, so the general procedures will carry over, making it easy to use.

This will be used throughout to create dataframes for fast processing and accessibilty of newly-obtained data.

In [6]:
import polars as pl

df = pl.DataFrame({'Date': event_dates , 'Event': event_names})

print(df)

shape: (24, 2)
┌───────────────────┬─────────────────────────────────────┐
│ Date              ┆ Event                               │
│ ---               ┆ ---                                 │
│ str               ┆ str                                 │
╞═══════════════════╪═════════════════════════════════════╡
│ August 17, 2024   ┆ UFC 305: Du Plessis vs. Adesanya    │
│ August 10, 2024   ┆ UFC Fight Night: Tybura vs. Spiv... │
│ August 03, 2024   ┆ UFC Fight Night: Sandhagen vs. N... │
│ July 27, 2024     ┆ UFC 304: Edwards vs. Muhammad 2     │
│ ...               ┆ ...                                 │
│ March 09, 2024    ┆ UFC 299: O'Malley vs. Vera 2        │
│ March 02, 2024    ┆ UFC Fight Night: Rozenstruik vs.... │
│ February 24, 2024 ┆ UFC Fight Night: Moreno vs. Royv... │
│ February 17, 2024 ┆ UFC 298: Volkanovski vs. Topuria    │
└───────────────────┴─────────────────────────────────────┘


Below is the SQL code written in python to migrate the data to the SQL server. As mentioned before, this is using SQL Server Management Studio (SSMS).
The code begins by creating a table called UFC_Events - this will be a list of all UFC events detailed on the ufc stats page online

In [19]:
# transfer data to SQL Server
# Convert the DataFrame to a CSV file
# df.write_csv('events.csv')
cursor = con.cursor()
# cursor.execute('''
#     CREATE TABLE UFC_Events (
#         Date VARCHAR(255),
#         Event VARCHAR(255)
#     )
# ''')
# cursor.commit()

# cursor.execute('''
#     BULK INSERT UFC_Events
#     FROM 'C:\\Users\\Aship\\PycharmProjects\\UFCDatabase\\DataCollection\\events.csv'
#     WITH (
#         FIELDTERMINATOR = ',',
#         ROWTERMINATOR = '\n',
#         FIRSTROW = 2
#     )
# ''')
# cursor.commit()

# cursor.execute('''
#     SELECT *
#     FROM UFC_Events
# ''')
cursor.commit()



In [21]:
cursor.execute('''
    SELECT *
    FROM UFC_Events
''')

for row in cursor.fetchall():
    print(row)

con.close()



('"August 17', ' 2024",UFC 305: Du Plessis vs. Adesanya')
('"August 10', ' 2024",UFC Fight Night: Tybura vs. Spivac 2')
('"August 03', ' 2024",UFC Fight Night: Sandhagen vs. Nurmagomedov')
('"July 27', ' 2024",UFC 304: Edwards vs. Muhammad 2')
('"July 20', ' 2024",UFC Fight Night: Lemos vs. Jandiroba')
('"July 13', ' 2024",UFC Fight Night: Namajunas vs. Cortez')
('"June 29', ' 2024",UFC 303: Pereira vs. Prochazka 2')
('"June 22', ' 2024",UFC Fight Night: Whittaker vs. Aliskerov')
('"June 15', ' 2024",UFC Fight Night: Perez vs. Taira')
('"June 08', ' 2024",UFC Fight Night: Cannonier vs. Imavov')
('"June 01', ' 2024",UFC 302: Makhachev vs. Poirier')
('"May 18', ' 2024",UFC Fight Night: Barboza vs. Murphy')
('"May 11', ' 2024",UFC Fight Night: Lewis vs. Nascimento')
('"May 04', ' 2024",UFC 301: Pantoja vs. Erceg')
('"April 27', ' 2024",UFC Fight Night: Nicolau vs. Perez')
('"April 13', ' 2024",UFC 300: Pereira vs. Hill')
('"April 06', ' 2024",UFC Fight Night: Allen vs. Curtis 2')
('"March

In [23]:
# reopens the connection
con = odbc.connect(connection_string)
cursor = con.cursor()
cursor.execute('''
    SELECT *
    FROM UFC_Events
''')

for row in cursor.fetchall():
    print(row)

con.close()



('"August 17', ' 2024",UFC 305: Du Plessis vs. Adesanya')
('"August 10', ' 2024",UFC Fight Night: Tybura vs. Spivac 2')
('"August 03', ' 2024",UFC Fight Night: Sandhagen vs. Nurmagomedov')
('"July 27', ' 2024",UFC 304: Edwards vs. Muhammad 2')
('"July 20', ' 2024",UFC Fight Night: Lemos vs. Jandiroba')
('"July 13', ' 2024",UFC Fight Night: Namajunas vs. Cortez')
('"June 29', ' 2024",UFC 303: Pereira vs. Prochazka 2')
('"June 22', ' 2024",UFC Fight Night: Whittaker vs. Aliskerov')
('"June 15', ' 2024",UFC Fight Night: Perez vs. Taira')
('"June 08', ' 2024",UFC Fight Night: Cannonier vs. Imavov')
('"June 01', ' 2024",UFC 302: Makhachev vs. Poirier')
('"May 18', ' 2024",UFC Fight Night: Barboza vs. Murphy')
('"May 11', ' 2024",UFC Fight Night: Lewis vs. Nascimento')
('"May 04', ' 2024",UFC 301: Pantoja vs. Erceg')
('"April 27', ' 2024",UFC Fight Night: Nicolau vs. Perez')
('"April 13', ' 2024",UFC 300: Pereira vs. Hill')
('"April 06', ' 2024",UFC Fight Night: Allen vs. Curtis 2')
('"March